In [61]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [62]:
data = pd.read_csv("Products.csv",sep = ';',encoding = 'unicode_escape')

In [63]:
print(data.shape)
data = data.drop_duplicates(keep = False).reset_index(drop=True)
data.shape

(19266, 6)


(18530, 6)

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18530 entries, 0 to 18529
Data columns (total 6 columns):
Barcode                18530 non-null int64
VendorDescription      18530 non-null object
VendorSubdepartment    18354 non-null object
FullProductName        18527 non-null object
CommonName             18527 non-null object
Location               18530 non-null object
dtypes: int64(1), object(5)
memory usage: 868.7+ KB


In [65]:
data = data.loc[pd.isnull(data.VendorSubdepartment) == False]

In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18354 entries, 0 to 18529
Data columns (total 6 columns):
Barcode                18354 non-null int64
VendorDescription      18354 non-null object
VendorSubdepartment    18354 non-null object
FullProductName        18354 non-null object
CommonName             18354 non-null object
Location               18354 non-null object
dtypes: int64(1), object(5)
memory usage: 1003.7+ KB


In [67]:
data.shape

(18354, 6)

In [68]:
x = data.values[:,2:5]
bag_of_words = []
for i in x:
    a = " "
    i[0] = "_".join(str(i[0]).split())
    a += i[0].lower()
    a+=" "
    i[1] = "_".join(str(i[1]).split())
    a+= i[1].lower()
    
    for j in str(i[2]).split(','):
        a+= " "
        j = "_".join(j.split())
        a+=str(j).lower()
#     a = a.strip().split()
#     a = list(set(a))
#     a = " ".join(a)
    a = " ".join(list(set(a.strip().split())))
    bag_of_words.append(a.strip())

In [69]:
len(bag_of_words)

18354

In [70]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
from tqdm import tqdm

In [72]:
count = CountVectorizer()
count_matrix = count.fit_transform(bag_of_words)

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(data.VendorDescription)
data1 = data.values
indices[:5]

0      AMB CORN ON THE COB
1         AMB TURNIP LOOSE
2    AMB WATERCRESS PUNNET
3    AMB WATERCRESS PUNNET
4              B&R MUSHRMS
Name: VendorDescription, dtype: object

In [73]:
count_matrix.todense().shape

(18354, 20619)

In [74]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.40824829, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.40824829, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.28867513,
        0.33333333],
       [0.        , 0.        , 0.        , ..., 0.28867513, 1.        ,
        0.8660254 ],
       [0.        , 0.        , 0.        , ..., 0.33333333, 0.8660254 ,
        1.        ]])

In [83]:
def return_vec(x,k):
    temp = []
    for i in x:
        temp.append(bag_of_words[i])
    temp = list(set(temp))
#     print(temp)
    temp =  [" ".join(temp)]
    temp = count.transform(temp)
    store = cosine_similarity(temp, count_matrix)
    
    print("Items bought:")
    for i in bought:
        print(data1[i][1],'\t\t\t',data1[i][2],'\t\t\t',data1[i][3])
    print()
    print()
    print()
    recomm = np.argsort(-store[0])
    print("Recommended Items:")
#     k = 10
    for i in range(len(recomm)):
        if not k:
            break
        if(recomm[i] in bought) or (store[0][recomm[i]]==0):
            continue
        print(data1[recomm[i]][1],'\t\t\t',data1[recomm[i]][2],'\t\t\t',data1[recomm[i]][3])
        k-=1

In [16]:
bought = [596,568,569]
return_vec(bought,10)

Items bought:
BOURSIN BLACK PEPPER 			 Cheese 			 Boursin Cheese Black Pepper
ARLA SKYR NRDC CHERRY 			 Yogurts & Desserts 			 Arle Sour Cherry Yogurt
ARLA SKYR STRAWBERRY 			 Yogurts & Desserts 			 Arla Strawberry Yogurt

Recommended Items:
WF STRAWBERRY YOGHURT 			 Yogurts & Desserts 			 Strawberry Yogurt
ONKEN WG SBRRY BIOPT 			 Yogurts & Desserts 			 Onken Wholegrain Strawberry Bio Yogurt
YEO VALLEY STRWBERRY 			 Yogurts & Desserts 			 Yeo Valley Strawberry Yogurt
SUCKIES STRAWBERRY 			 Yogurts & Desserts 			 Strawberry Suckie
DANONE OYKOS SBERRY PACK 			 Yogurts & Desserts 			 Oykos Strawberry Yogurt
MLLR LGHT STRWBRY 			 Yogurts & Desserts 			 Muller Light Strawberry Yogurt
MLLR LGHT GRK STRWB PACK 			 Yogurts & Desserts 			 Muller Light Greek Strawberry Yogurt
MLLR CRNR FRUIT STRWBRY 			 Yogurts & Desserts 			 Muller Corner Strawberry Yogurt
MLLR CRNR BLISS SBERRY 4 			 Yogurts & Desserts 			 Muller Corner Strawberry Bliss
NOMADIC STRWB OAT CLUSTER 			 Yogurts & Desserts 			 Str

In [87]:
bought = [596,568,23,34,569,4577,7899,12345,4565]
return_vec(bought,20)

Items bought:
BOURSIN BLACK PEPPER 			 Cheese 			 Boursin Cheese Black Pepper
ARLA SKYR NRDC CHERRY 			 Yogurts & Desserts 			 Arle Sour Cherry Yogurt
CF ATHENRY OAT F/J 			 Health Foods 			 Flapjacks
CLF FLAX CHIA APPL&CINNAMON 			 Health Foods 			 Milled Flaxseed, Chia, Apple & Cinnamon
ARLA SKYR STRAWBERRY 			 Yogurts & Desserts 			 Arla Strawberry Yogurt
SHAPE FIT PREGNANCY 			 NEWS & MAGS 			 Shape Fit Pregnancy
COLOUR 			 Newspapers 			 Magazine
KH 2017 CALENDARS 			 Seasonal Non Food 			 Calendar
ANGELINA 			 Newspapers 			 Magazine



Recommended Items:
STAR WARS ADVENTURES 			 Newspapers 			 Magazine
THUNDERBIRDS ARE GO 			 Newspapers 			 Magazine
TRUE LIFE SPECIAL 			 Newspapers 			 Magazine
PUZZLELIFE 			 Newspapers 			 Magazine
BUILD IT 			 Newspapers 			 Magazine
PUZZLE WORLD 			 Newspapers 			 Magazine
MY LITT PONY SPL MOS PW 			 Newspapers 			 Magazine
SUPERFOOD 			 Newspapers 			 Magazine
SUNDAY SUN 			 Newspapers 			 Magazine
FROZEN 			 Newspapers 			 Magazine
MR TUMBL